In [1]:
import urllib3
from bs4 import BeautifulSoup
import requests
import certifi
import time
import datetime

In [2]:
#!/usr/bin/python
import MySQLdb

db = MySQLdb.connect(host="localhost",    # your host, usually localhost
                     user="root",         # your username
                     passwd="root",  # your password
                     db="pipa_db")        # name of the data base

cur = db.cursor()

In [3]:
show_tables = "show tables"
cur.execute(show_tables)

6L

In [4]:
dailyMax = 1000
batchNum = 20
sleeptimeforPubmed =5
backstep =20

cur.execute('select pmid from last_pmid')
endingPMIDtuple = max(cur.fetchall())
endingPMID = endingPMIDtuple[0]
startingPMID=28315600 #endingPMID -backstep
print endingPMID, startingPMID
    

28315622 28315600


In [5]:
cur.execute('truncate articles')
# erase data or re-create articles table

0L

In [6]:
%%time
        for i in range (startingPMID, startingPMID+dailyMax, batchNum):
            PMIDs = ""

            for j in range (batchNum):
                PMIDs = PMIDs + str(i + j ) +","

            url = "https://www.ncbi.nlm.nih.gov/pubmed/"+ PMIDs +"?report=medline&format=text" 

            #time.sleep(sleeptimeforPubmed)
            http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())
            r = http.request('GET', url)
            soup = BeautifulSoup(r.data, "lxml")

            allInfo = unicode(soup.pre.string)

            if allInfo.find('PMID- ')<0:
                print url
                print 'bad end'        
                break

            while allInfo.find('AB  -')>0:
                IdIndexStart = allInfo.index('PMID- ')+6
                IdIndexEnd = allInfo.index('\n', IdIndexStart)
                PMID = allInfo[IdIndexStart:IdIndexEnd]

                tiIndexStart = allInfo.index('TI  -')+6
                if allInfo.find(  'PG  -', tiIndexStart)>0:
                    tiIndexEnd =   allInfo.index('PG  -', tiIndexStart)-1
                elif allInfo.find('LID -', tiIndexStart)>0:
                    tiIndexEnd =   allInfo.index('LID -', tiIndexStart)-1
                else:
                    tiIndexEnd =   allInfo.index('AB  -', tiIndexStart)-1
                ti = allInfo[tiIndexStart:tiIndexEnd]
                
                abIndexStart = allInfo.index('AB  -')+6
                abIndexEnd =   allInfo.index('FAU -', abIndexStart)-1
                ab = ti+"        Abstract:"+allInfo[abIndexStart:abIndexEnd]

                query = "insert into articles (pmid, abstract) values ("+ PMID +", '"+ ab + "')"
                endingPMID = int(PMID)
                if (endingPMID%10==0):
                    print endingPMID

                try:
                    cur.execute(query)
                    db.commit()

                except:
                    db.rollback()

                allInfo = allInfo[abIndexEnd:]

        now = datetime.datetime.now()       
        query = 'insert into last_pmid (record_date, pmid) values ("%s", %d)'%(now, endingPMID)
        cur.execute(query)
        db.commit()
# write endingPMID to db
db.close()    
    

28315600
28315610
28315620
https://www.ncbi.nlm.nih.gov/pubmed/28315640,28315641,28315642,28315643,28315644,28315645,28315646,28315647,28315648,28315649,28315650,28315651,28315652,28315653,28315654,28315655,28315656,28315657,28315658,28315659,?report=medline&format=text
bad end
CPU times: user 67.4 ms, sys: 7.65 ms, total: 75.1 ms
Wall time: 1.69 s
